# Global Lesioning Analysis

Note: The first time this is run (i.e., with an empty cache), it will
take 2-3 hours from beginning to end.

In [1]:
import textwrap
import os
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from prep_data import \
    mazes,\
    model_preds,\
    get_exp1_at,\
    get_exp2_at,\
    get_exp3_at__exp3_mt,\
    get_exp4a_ht,\
    get_exp4b_ht
from analysisutils import short_predictor_names
import analysisutils

from vgc_project.r import create_R_model_interface, ImmutableDataFrame
joblib_cache_location = "./_analysiscache"
rmods = create_R_model_interface(joblib_cache_location=joblib_cache_location)


# Dynamic

In [2]:
predictors = [
    'dynamic_vgc_weight_Z',
    'log_traj_based_hitcount_Z',
    'graph_based_hitcount_Z',
    'bottleneck_dist_Z',
    'sr_occ_Z',
    'nav_mindist_Z',
    'nav_mindist_timestep_Z',
    'optpolicy_dist_Z',
    'goal_dist_Z',
    'start_dist_Z',
    'walls_dist_Z',
    'center_dist_Z',
]
exp_analysis_predictors = dict(
    exp1=[p for p in predictors if "optpolicy_" not in p],
    exp2_early=[p for p in predictors if "nav_mindist" not in p],
    exp2_full=[p for p in predictors if "optpolicy_" not in p],
    exp3=[p for p in predictors if "optpolicy_" not in p],
    exp4=[p for p in predictors if "nav_mindist" not in p],
)
    


In [3]:
exp1_at = get_exp1_at()
exp1_at_im = ImmutableDataFrame(exp1_at)
exp1_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp1']),
    data=exp1_at_im
)
exp1_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 1 Accuracy",
    data=exp1_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp1'],
    rmods=rmods
)

exp2_at = get_exp2_at()
exp2_early_at_im = ImmutableDataFrame(exp2_at[exp2_at['earlyterm'] == 'earlyterm'])
exp2_full_at_im = ImmutableDataFrame(exp2_at[exp2_at['earlyterm'] == 'full'])
exp2_at_im = ImmutableDataFrame(exp2_at)
exp2_full_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp2_full']),
    data=exp2_full_at_im
)
exp2_early_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp2_early']),
    data=exp2_early_at_im
)
exp2_early_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 2 Accuracy Early",
    data=exp2_early_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp2_early'],
    rmods=rmods
)
exp2_full_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 2 Accuracy Full",
    data=exp2_full_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp2_full'],
    rmods=rmods
)


exp3_at, exp3_mt = get_exp3_at__exp3_mt()
exp3_at_im = ImmutableDataFrame(exp3_at)
exp3_mt_im = ImmutableDataFrame(exp3_mt)
exp3_correct_global_model = rmods.lmer(
    formula="correct ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp3']),
    data=exp3_mt_im
)
exp3_conf_global_model = rmods.lmer(
    formula="conf_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp3']),
    data=exp3_mt_im
)
exp3_attn_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp3']),
    data=exp3_at_im
)
exp3_correct_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 3 Accuracy",
    data=exp3_mt_im,
    dv="correct",
    model_func="glmer",
    random_effects="(1 | sessionId)",
    predictors=exp_analysis_predictors['exp3'],
    rmods=rmods
)
exp3_conf_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 3 Confidence",
    data=exp3_mt_im,
    dv="conf_N",
    model_func="lmer",
    random_effects="(1 | sessionId)",
    predictors=exp_analysis_predictors['exp3'],
    rmods=rmods
)
exp3_attn_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 3 Awareness",
    data=exp3_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId)",
    predictors=exp_analysis_predictors['exp3'],
    rmods=rmods
)


exp4a_ht = get_exp4a_ht()
exp4b_ht = get_exp4b_ht()
exp4a_ht_im = ImmutableDataFrame(exp4a_ht)
exp4b_ht_im = ImmutableDataFrame(exp4b_ht)
exp4a_hov_global_model = rmods.glmer(
    formula="hovered ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp4']),
    data=exp4a_ht_im
)
exp4a_dur_global_model = rmods.lmer(
    formula="log_hoverduration ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp4']),
    data=exp4a_ht_im
)
exp4b_hov_global_model = rmods.glmer(
    formula="hovered ~ (1 | sessionId) + (1 | grid) +"+"+".join(exp_analysis_predictors['exp4']),
    data=exp4b_ht_im
)
exp4b_dur_global_model = rmods.lmer(
    formula="log_hoverduration ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp4']),
    data=exp4b_ht_im
)
exp4a_hov_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4a Hover",
    data=exp4a_ht_im,
    dv="hovered",
    model_func="glmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)
exp4a_dur_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4a Hover Duration",
    data=exp4a_ht_im,
    dv="log_hoverduration",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)
exp4b_hov_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4b Hover",
    data=exp4b_ht_im,
    dv="hovered",
    model_func="glmer",
    random_effects="(1 | sessionId)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)
exp4b_dur_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4b Hover Duration",
    data=exp4b_ht_im,
    dv="log_hoverduration",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)

Loading Experiment 1 Attention Trials


100%|██████████| 11/11 [00:02<00:00,  4.48it/s]


## Tables with all estimated coefficients

In [4]:
def format_coefficients(mod):
    return pd.DataFrame(mod['coefficients']).\
        set_index('effect').\
        rename(columns={"estimate": r"$\beta$", "se": "SE"})
all_global_models = [
    dict(model=exp1_global_model, name=("Initial Exp", "Awareness")),
    dict(model=exp2_full_global_model, name=("Up-front Exp", "Awareness (F)")),
    dict(model=exp2_early_global_model, name=("Up-front Exp","Awareness (E)")),
    dict(model=exp3_correct_global_model, name=("Critical Mazes Exp", "Accuracy")),
    dict(model=exp3_conf_global_model, name=("Critical Mazes Exp", "Confidence")),
    dict(model=exp3_attn_global_model, name=("Critical Mazes Exp", "Awareness")),
    dict(model=exp4a_hov_global_model, name=("Process-Tracing Exp (Initial Mazes)", "Hovering")),
    dict(model=exp4a_dur_global_model, name=("Process-Tracing Exp (Initial Mazes)", "Duration")),
    dict(model=exp4b_hov_global_model, name=("Process-Tracing Exp (Critical Mazes)", "Hovering")),
    dict(model=exp4b_dur_global_model, name=("Process-Tracing Exp (Critical Mazes)", "Duration")),
]
coeff_table = pd.concat(
    [format_coefficients(mod['model']) for mod in all_global_models],
    axis=1,
    keys=[mod['name'] for mod in all_global_models]
)
coeff_table = coeff_table.reindex(["(Intercept)"]+predictors)
coeff_table.index = ["Intercept" if "Intercept" in i else short_predictor_names[i.replace("_Z", "")] for i in coeff_table.index.values]

In [5]:
coeff_table1_3 = coeff_table[["Initial Exp", "Up-front Exp", "Critical Mazes Exp"]]
coeff_string1_3 = coeff_table1_3.round(3).to_latex(
    na_rep="---", 
    escape=False,
    multicolumn_format='c',
    column_format="lrrrrrrrrrrrr",
    col_space=1,
)
coeff_string1_3 = coeff_string1_3.split('\n')

coeff_string1_3.insert(
    4, r"\cmidrule(lr){2-3} \cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11} \cmidrule(lr){12-13}"
)
coeff_string1_3.insert(
    3, r"\cmidrule(lr){2-3} \cmidrule(lr){5-6} \cmidrule(lr){9-12}"
)
coeff_string1_3 = "\n".join(coeff_string1_3)
with open("results/coeff_table1_3.tex", "w") as file:
    file.write(
        textwrap.dedent(r"""
        \documentclass{standalone}
        \usepackage{booktabs}
        \usepackage{tabularx}
        \usepackage{caption}
        \usepackage[scaled=0.7025996185887785]{helvet}
        \linespread{0.7025996185887785}
        \renewcommand{\familydefault}{\sfdefault}
        \begin{document}
        \fontsize{7}{8.4}
        """) + \
        coeff_string1_3 + \
        textwrap.dedent(r"""
        \end{document}
        """)
    )
# with open("results/coeff_table1_3.tex", "w") as file:
#     file.write(
#         textwrap.dedent(r"""
#         \documentclass{standalone}
#         \usepackage{booktabs}
#         \usepackage{tabularx}
#         \renewcommand{\familydefault}{\sfdefault}
#         \begin{document}
#         \fontsize{7}{8.4}
#         \selectfont
#         """) + \
#         coeff_string1_3 + \
#         textwrap.dedent(r"""
#         \end{document}
#         """)
#     )

# Create pdf to combine in vector image editor
# process = subprocess.Popen("pdflatex -output-directory results results/coeff_table1_3.tex".split())
process = subprocess.Popen("xelatex -output-directory results results/coeff_table1_3.tex".split())
output, error = process.communicate()
print(output, error)
coeff_table1_3

None None


Initial Exp            Up-front Exp                          \
                Awareness           Awareness (F)           Awareness (E)   
                  $\beta$        SE       $\beta$        SE       $\beta$   
Intercept        0.529909  0.016803      0.324723  0.018979      0.493851   
VGC              0.085432  0.003776      0.095463  0.005613      0.080813   
Traj HS          0.000432  0.005056      0.002681  0.007574      0.014976   
Graph HS         0.028512  0.004961      0.012401  0.007344      0.005116   
Bottleneck      -0.028361  0.003351     -0.044047  0.005041     -0.051698   
SR Overlap      -0.030578  0.003101     -0.035565  0.004605     -0.037296   
Nav Dist        -0.114301  0.003400     -0.277118  0.012018           NaN   
Nav Dist Step    0.014369  0.003831     -0.027373  0.003462           NaN   
Opt Dist              NaN       NaN           NaN       NaN     -0.084397   
Goal Dist       -0.006556  0.005076      0.004553  0.007336     -0.005735   
Start Dist      -0.020677  0.004709      0.016669  0.006934     -0.022616   
Wall Dist       -0.237331  0.024137     -0.153150  0.037299     -0.190602   
Center Dist      0.237912  0.024668      0.182077  0.038124      0.213590   

                        Critical Mazes Exp                                 \
                                  Accuracy           Confidence             
                     SE            $\beta$        SE    $\beta$        SE   
Intercept      0.015761           0.560405  0.065782   0.643596  0.041231   
VGC            0.005572           0.149984  0.014693   0.118076  0.008637   
Traj HS        0.007748           0.026279  0.029648  -0.011134  0.017429   
Graph HS       0.007463          -0.034318  0.034395  -0.036385  0.020220   
Bottleneck     0.005167           0.004195  0.013749  -0.012529  0.008085   
SR Overlap     0.004758          -0.033163  0.045788   0.003732  0.026961   
Nav Dist            NaN          -0.034779  0.012005  -0.009015  0.007069   
Nav Dist Step       NaN          -0.008735  0.010479  -0.007456  0.006265   
Opt Dist       0.005318                NaN       NaN        NaN       NaN   
Goal Dist      0.007451           0.116164  0.072909  -0.094899  0.042881   
Start Dist     0.006984           0.069317  0.068685  -0.125606  0.040392   
Wall Dist      0.036797          -0.039331  0.068408  -0.094800  0.040226   
Center Dist    0.037588           0.019026  0.057631   0.114229  0.033880   

                                   
              Awareness            
                $\beta$        SE  
Intercept      0.428847  0.043849  
VGC            0.211752  0.009209  
Traj HS        0.013275  0.018504  
Graph HS      -0.042383  0.021390  
Bottleneck    -0.033988  0.008545  
SR Overlap     0.086030  0.028466  
Nav Dist      -0.063943  0.007359  
Nav Dist Step -0.042880  0.006570  
Opt Dist            NaN       NaN  
Goal Dist      0.082502  0.045642  
Start Dist    -0.014215  0.042982  
Wall Dist      0.066204  0.042878  
Center Dist    0.007152  0.036181

In [6]:
!xelatex -output-directory results results/coeff_table1_3.tex

This is XeTeX, Version 3.14159265-2.6-0.99992 (TeX Live 2015) (preloaded format=xelatex)
 restricted \write18 enabled.
entering extended mode
(./results/coeff_table1_3.tex
LaTeX2e <2015/01/01>
Babel <3.9l> and hyphenation patterns for 79 languages loaded.
(/usr/local/texlive/2015/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2012/09/15 v1.1b Class to compile TeX sub-files stan
dalone
(/usr/local/texlive/2015/texmf-dist/tex/generic/oberdiek/ifluatex.sty)
(/usr/local/texlive/2015/texmf-dist/tex/generic/oberdiek/ifpdf.sty)
(/usr/local/texlive/2015/texmf-dist/tex/generic/ifxetex/ifxetex.sty)
(/usr/local/texlive/2015/texmf-dist/tex/latex/xkeyval/xkeyval.sty
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/xkeyval.tex
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/xkvutils.tex
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/keyval.tex))))
(/usr/local/texlive/2015/texmf-dist/tex/latex/standalone/standalone.cfg)
(/usr/local/texlive/2015/texmf

In [7]:
coeff_table4 = coeff_table[["Process-Tracing Exp (Initial Mazes)", "Process-Tracing Exp (Critical Mazes)"]].dropna(axis=0)
coeff_string4 = coeff_table4.round(3).to_latex(
        na_rep="---", 
        escape=False,
        multicolumn_format='c',
        column_format="lrrrrrrrrrrrr",
        col_space=1,
    )
coeff_string4 = coeff_string4.split('\n')

coeff_string4.insert(
    4, r"\cmidrule(lr){2-3} \cmidrule(lr){4-5} \cmidrule(lr){6-7} \cmidrule(lr){8-9}"
)
coeff_string4.insert(
    3, r"\cmidrule(lr){2-5} \cmidrule(lr){6-9}"
)
coeff_string4 = "\n".join(coeff_string4)
# with open("results/coeff_table4.tex", "w") as file:
#     file.write(
#         textwrap.dedent(r"""
#         \documentclass{standalone}
#         \usepackage{booktabs}
#         \usepackage{tabularx}
#         \renewcommand{\familydefault}{\sfdefault}
#         \begin{document}
#         \fontsize{7}{8.4}
#         \selectfont
#         """) + \
#         coeff_string4 + \
#         textwrap.dedent(r"""
#         \end{document}
#         """)
#     )
with open("results/coeff_table4.tex", "w") as file:
    file.write(
        textwrap.dedent(r"""
        \documentclass{standalone}
        \usepackage{booktabs}
        \usepackage{tabularx}
        \usepackage{caption}
        \usepackage[scaled=0.7025996185887785]{helvet}
        \linespread{0.7025996185887785}
        \renewcommand{\familydefault}{\sfdefault}
        \begin{document}
        \fontsize{7}{8.4}
        """) + \
        coeff_string4 + \
        textwrap.dedent(r"""
        \end{document}
        """)
    )
    
# Create pdf to combine in vector image editor
# process = subprocess.Popen("pdflatex -output-directory results results/coeff_table4.tex".split())
process = subprocess.Popen("xelatex -output-directory results results/coeff_table4.tex".split())
output, error = process.communicate()
print(output, error)
coeff_table4

None None


Process-Tracing Exp (Initial Mazes)                                \
                                       Hovering            Duration             
                                        $\beta$        SE   $\beta$        SE   
Intercept                              0.716582  0.128319  6.515188  0.071131   
VGC                                    0.587996  0.035015  0.233812  0.017448   
Traj HS                                0.452349  0.045988  0.101649  0.028082   
Graph HS                              -0.195891  0.044550 -0.088999  0.023827   
Bottleneck                            -0.223951  0.031113 -0.175371  0.016230   
SR Overlap                            -0.017182  0.028100  0.024138  0.015583   
Opt Dist                              -0.402155  0.031524 -0.022478  0.019591   
Goal Dist                             -0.146495  0.044790  0.024968  0.023779   
Start Dist                            -0.330845  0.042450  0.047508  0.022419   
Wall Dist                             -0.986641  0.206861 -0.907574  0.116933   
Center Dist                            1.293709  0.211266  1.142729  0.119101   

            Process-Tracing Exp (Critical Mazes)                                
                                        Hovering            Duration            
                                         $\beta$        SE   $\beta$        SE  
Intercept                               0.544450  0.319804  5.886750  0.181496  
VGC                                     0.702266  0.054027  0.480561  0.029781  
Traj HS                                 0.196713  0.134384  0.351523  0.079551  
Graph HS                                0.394741  0.141104 -0.311264  0.083817  
Bottleneck                              0.058286  0.059355 -0.056988  0.030471  
SR Overlap                              1.358678  0.221383  0.388377  0.105662  
Opt Dist                                0.329169  0.053822  0.147376  0.031585  
Goal Dist                              -1.036686  0.333826  0.070516  0.189523  
Start Dist                             -1.617707  0.311145 -0.190201  0.182416  
Wall Dist                              -0.145171  0.251776 -0.339270  0.147441  
Center Dist                             1.282434  0.188821  0.817221  0.127296

## Create lesioned table

In [8]:
all_lesioning_res = [
    dict(res=exp1_lesioning_res, name=("Initial Exp", "Awareness")),
    dict(res=exp2_full_lesioning_res, name=("Up-front Exp", "Awareness (F)")),
    dict(res=exp2_early_lesioning_res, name=("Up-front Exp","Awareness (E)")),
    dict(res=exp3_correct_lesioning_res, name=("Critical Mazes Exp", "Accuracy")),
    dict(res=exp3_conf_lesioning_res, name=("Critical Mazes Exp", "Confidence")),
    dict(res=exp3_attn_lesioning_res, name=("Critical Mazes Exp", "Awareness")),
    dict(res=exp4a_hov_lesioning_res, name=("Process-Tracing Exp (Initial Mazes)", "Hovering")),
    dict(res=exp4a_dur_lesioning_res, name=("Process-Tracing Exp (Initial Mazes)", "Duration")),
    dict(res=exp4b_hov_lesioning_res, name=("Process-Tracing Exp (Critical Mazes)", "Hovering")),
    dict(res=exp4b_dur_lesioning_res, name=("Process-Tracing Exp (Critical Mazes)", "Duration")),
]

### Likelihood ratio test summary

In [9]:
all_summaries = []
for mod in all_lesioning_res:
    all_summaries.append(
        f"{', '.join(mod['name'])}: {mod['res'].likelihood_ratio_test('dynamic_vgc_weight_Z', 1)}"
    )
all_summaries = '; \n'.join(all_summaries)
with open("./inputs/all_plan_exp_vgc_global_lesion_llr_dvgc.tex", "w") as file:
    file.write(all_summaries)
print(all_summaries)

Initial Exp, Awareness: $\chi^2(1) = 501.11, p < 1.0 \times 10^{-16}$; 
Up-front Exp, Awareness (F): $\chi^2(1) = 282.17, p < 1.0 \times 10^{-16}$; 
Up-front Exp, Awareness (E): $\chi^2(1) = 206.14, p < 1.0 \times 10^{-16}$; 
Critical Mazes Exp, Accuracy: $\chi^2(1) = 114.87, p < 1.0 \times 10^{-16}$; 
Critical Mazes Exp, Confidence: $\chi^2(1) = 181.28, p < 1.0 \times 10^{-16}$; 
Critical Mazes Exp, Awareness: $\chi^2(1) = 486.99, p < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Initial Mazes), Hovering: $\chi^2(1) = 294.40, p < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Initial Mazes), Duration: $\chi^2(1) = 177.58, p < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Critical Mazes), Hovering: $\chi^2(1) = 183.52, p < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Critical Mazes), Duration: $\chi^2(1) = 251.16, p < 1.0 \times 10^{-16}$


In [10]:
def format_global_lesioning_res(res):
    return \
        res[['dAIC.FULL', 'lesioned_effect']].\
        set_index('lesioned_effect')["dAIC.FULL"].drop('FULL')

global_lesion_table = pd.concat(
    [format_global_lesioning_res(res['res']) for res in all_lesioning_res],
    axis=1,
    keys=[res['name'] for res in all_lesioning_res]
)


global_lesion_table = global_lesion_table.reindex(predictors)
global_lesion_table.index = ["Intercept" if "Intercept" in i else short_predictor_names[i.replace("_Z", "")] for i in global_lesion_table.index.values]

# with annotation
col_ranks = global_lesion_table.rank(axis=0, ascending=False)
annot_global_lesion_table = []
for model, row in global_lesion_table.iterrows():
    annot_row = []
    for measure, daic in row.to_dict().items():
        if np.isnan(daic):
            annot_row.append("---")
            continue
            
        daic_rank = col_ranks.loc[model, measure]
        if daic_rank == 1:
            annot = "$^*$"
        elif daic_rank == 2:
            annot = r"\textsuperscript{\textdagger}"
        else:
            annot = ""
        annot_row.append(str(int(daic))+annot)
    annot_global_lesion_table.append(annot_row)

annot_global_lesion_table = pd.DataFrame(annot_global_lesion_table, index=global_lesion_table.index, columns=global_lesion_table.columns)
global_lesion_string = annot_global_lesion_table.to_latex(
    column_format="lcccccccccc",
    col_space=1,
    escape=False,
    # caption=
)

# without annotation
# global_lesion_string = global_lesion_table.round(0).to_latex(
#     na_rep="---", 
#     float_format=lambda f: str(int(f)) if not np.isnan(f) else "---",
#     column_format="lcccccccccc",
#     col_space=1,
#     escape=False
# )


global_lesion_string = global_lesion_string.replace(
    r"{} &                       Initial Exp & \multicolumn{2}{l}{Up-front Exp} & \multicolumn{3}{l}{Critical Mazes Exp} & \multicolumn{2}{l}{Process-Tracing Exp (Initial Mazes)} & \multicolumn{2}{l}{Process-Tracing Exp (Critical Mazes)} \\",
    r"{} & Initial Exp & \multicolumn{2}{c}{Up-front Exp} & \multicolumn{3}{c}{Critical Mazes Exp} & \multicolumn{2}{>{\centering}m{2.5cm}}{Process-Tracing Exp (Initial Mazes)} & \multicolumn{2}{>{\centering}m{2.5cm}}{Process-Tracing Exp (Critical Mazes)} \\" + \
    "\n" + r"\cmidrule(lr){2-2}\cmidrule(lr){3-4}\cmidrule(lr){5-7}\cmidrule(lr){8-9}\cmidrule(lr){10-11}",
)

global_lesion_string = global_lesion_string.replace(
    r"\bottomrule",
    r"\bottomrule"+"\n"+r"\multicolumn{10}{l}{$^*$Largest increase in AIC after lesioning; \textsuperscript{\textdagger}Second-largest increase}"
)

with open("results/global_lesion_table.tex", "w") as file:
    file.write(
        textwrap.dedent(r"""
        \documentclass{standalone}
        \usepackage{booktabs}
        \usepackage{tabularx}
        \usepackage{caption}
        \usepackage[scaled=0.7025996185887785]{helvet}
        \linespread{0.7025996185887785}
        \renewcommand{\familydefault}{\sfdefault}
        \begin{document}
        \fontsize{7}{8.4}
        """) + \
        global_lesion_string + \
        textwrap.dedent(r"""
        \end{document}
        """)
    )

# Create pdf to combine in vector image editor
# need to figure out how to make text in right font and also recognizable as text in eps
process = subprocess.Popen("xelatex -output-directory results results/global_lesion_table.tex".split())
output, error = process.communicate()
print(output, error)
global_lesion_table

None None


Initial Exp  Up-front Exp               Critical Mazes Exp  \
                 Awareness Awareness (F) Awareness (E)           Accuracy   
VGC             499.113392    280.172151    204.136622         112.874654   
Traj HS          -1.992742     -1.877598      1.714661          -1.753345   
Graph HS         30.957267      0.849995     -1.531974          -1.516601   
Bottleneck       69.339467     72.877370     96.951598          -1.842230   
SR Overlap       93.130779     55.225527     57.965438          -1.418159   
Nav Dist       1080.062002    508.904183           NaN           6.199656   
Nav Dist Step    12.027048     59.455131           NaN          -1.210297   
Opt Dist               NaN           NaN    244.477787                NaN   
Goal Dist        -0.331860     -1.615811     -1.411320           1.355383   
Start Dist       17.244173      3.774450      8.434661          -0.319641   
Wall Dist        94.130729     14.811883     24.773599          -1.990271   
Center Dist      90.476943     20.716570     30.208532          -1.854504   

                                      Process-Tracing Exp (Initial Mazes)  \
               Confidence   Awareness                            Hovering   
VGC            179.277829  484.994809                          292.395320   
Traj HS         -1.591931   -1.485398                           95.092440   
Graph HS         1.236459    1.923610                           17.404013   
Bottleneck       0.400619   13.777530                           49.971014   
SR Overlap      -1.980839    7.120009                           -1.627211   
Nav Dist        -0.374190   72.565281                                 NaN   
Nav Dist Step   -0.584985   40.296243                                 NaN   
Opt Dist              NaN         NaN                          163.797798   
Goal Dist        2.893907    1.265464                            8.714132   
Start Dist       7.655055   -1.890630                           59.504010   
Wall Dist        3.548639    0.383043                           20.330180   
Center Dist      9.345966   -1.960925                           34.608521   

                          Process-Tracing Exp (Critical Mazes)              
                 Duration                             Hovering    Duration  
VGC            175.583795                           181.520634  249.160345  
Traj HS         10.835063                            13.664664   13.730553  
Graph HS        11.937308                             4.717786    9.855682  
Bottleneck     113.404433                             2.248030    1.304701  
SR Overlap       0.398225                            94.952841   10.994399  
Nav Dist              NaN                                  NaN         NaN  
Nav Dist Step         NaN                                  NaN         NaN  
Opt Dist        -0.692228                            15.740069   19.267673  
Goal Dist       -0.921900                             9.540669   -1.871694  
Start Dist       2.370825                            37.358712   -1.015470  
Wall Dist       57.927243                            -1.825581    3.130438  
Center Dist     89.429743                            39.817174   38.170494

In [11]:
!xelatex -output-directory results results/global_lesion_table.tex

This is XeTeX, Version 3.14159265-2.6-0.99992 (TeX Live 2015) (preloaded format=xelatex)
 restricted \write18 enabled.
entering extended mode
(./results/global_lesion_table.tex
LaTeX2e <2015/01/01>
Babel <3.9l> and hyphenation patterns for 79 languages loaded.
(/usr/local/texlive/2015/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2012/09/15 v1.1b Class to compile TeX sub-files stan
dalone
(/usr/local/texlive/2015/texmf-dist/tex/generic/oberdiek/ifluatex.sty)
(/usr/local/texlive/2015/texmf-dist/tex/generic/oberdiek/ifpdf.sty)
(/usr/local/texlive/2015/texmf-dist/tex/generic/ifxetex/ifxetex.sty)
(/usr/local/texlive/2015/texmf-dist/tex/latex/xkeyval/xkeyval.sty
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/xkeyval.tex
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/xkvutils.tex
(/usr/local/texlive/2015/texmf-dist/tex/generic/xkeyval/keyval.tex))))
(/usr/local/texlive/2015/texmf-dist/tex/latex/standalone/standalone.cfg)
(/usr/local/texlive/2015/

In [12]:
annot_global_lesion_table.to_latex()

'\\begin{tabular}{lllllllllll}\n\\toprule\n{} &                       Initial Exp & \\multicolumn{2}{l}{Up-front Exp} & \\multicolumn{3}{l}{Critical Mazes Exp} & \\multicolumn{2}{l}{Process-Tracing Exp (Initial Mazes)} & \\multicolumn{2}{l}{Process-Tracing Exp (Critical Mazes)} \\\\\n{} &                         Awareness &                     Awareness (F) &                     Awareness (E) &                        Accuracy &                      Confidence &                        Awareness &                            Hovering &                          Duration &                             Hovering &                         Duration \\\\\n\\midrule\nVGC           &  499\\textbackslash textsuperscript\\{\\textbackslash textdagger\\} &  280\\textbackslash textsuperscript\\{\\textbackslash textdagger\\} &  204\\textbackslash textsuperscript\\{\\textbackslash textdagger\\} &                         112\\$\\textasciicircum *\\$ &                         179\\$\\textasciicircum *\\$ & 

# Static

In [13]:
predictors = [
    'static_vgc_weight_Z',
    'log_traj_based_hitcount_Z',
    'graph_based_hitcount_Z',
    'bottleneck_dist_Z',
    'sr_occ_Z',
    'nav_mindist_Z',
    'nav_mindist_timestep_Z',
    'optpolicy_dist_Z',
    'goal_dist_Z',
    'start_dist_Z',
    'walls_dist_Z',
    'center_dist_Z',
]
exp_analysis_predictors = dict(
    exp1=[p for p in predictors if "optpolicy_" not in p],
    exp2_early=[p for p in predictors if "nav_mindist" not in p],
    exp2_full=[p for p in predictors if "optpolicy_" not in p],
    exp3=[p for p in predictors if "optpolicy_" not in p],
    exp4=[p for p in predictors if "nav_mindist" not in p],
)
    


In [14]:
exp1_at = get_exp1_at()
exp1_at_im = ImmutableDataFrame(exp1_at)
exp1_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp1']),
    data=exp1_at_im
)
exp1_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 1 Accuracy",
    data=exp1_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp1'],
    rmods=rmods
)

exp2_at = get_exp2_at()
exp2_early_at_im = ImmutableDataFrame(exp2_at[exp2_at['earlyterm'] == 'earlyterm'])
exp2_full_at_im = ImmutableDataFrame(exp2_at[exp2_at['earlyterm'] == 'full'])
exp2_at_im = ImmutableDataFrame(exp2_at)
exp2_full_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp2_full']),
    data=exp2_full_at_im
)
exp2_early_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp2_early']),
    data=exp2_early_at_im
)
exp2_early_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 2 Accuracy Early",
    data=exp2_early_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp2_early'],
    rmods=rmods
)
exp2_full_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 2 Accuracy Full",
    data=exp2_full_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp2_full'],
    rmods=rmods
)


exp3_at, exp3_mt = get_exp3_at__exp3_mt()
exp3_at_im = ImmutableDataFrame(exp3_at)
exp3_mt_im = ImmutableDataFrame(exp3_mt)
exp3_correct_global_model = rmods.lmer(
    formula="correct ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp3']),
    data=exp3_mt_im
)
exp3_conf_global_model = rmods.lmer(
    formula="conf_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp3']),
    data=exp3_mt_im
)
exp3_attn_global_model = rmods.lmer(
    formula="attention_N ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp3']),
    data=exp3_at_im
)
exp3_correct_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 3 Accuracy",
    data=exp3_mt_im,
    dv="correct",
    model_func="glmer",
    random_effects="(1 | sessionId)",
    predictors=exp_analysis_predictors['exp3'],
    rmods=rmods
)
exp3_conf_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 3 Confidence",
    data=exp3_mt_im,
    dv="conf_N",
    model_func="lmer",
    random_effects="(1 | sessionId)",
    predictors=exp_analysis_predictors['exp3'],
    rmods=rmods
)
exp3_attn_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 3 Awareness",
    data=exp3_at_im,
    dv="attention_N",
    model_func="lmer",
    random_effects="(1 | sessionId)",
    predictors=exp_analysis_predictors['exp3'],
    rmods=rmods
)


exp4a_ht = get_exp4a_ht()
exp4b_ht = get_exp4b_ht()
exp4a_ht_im = ImmutableDataFrame(exp4a_ht)
exp4b_ht_im = ImmutableDataFrame(exp4b_ht)
exp4a_hov_global_model = rmods.glmer(
    formula="hovered ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp4']),
    data=exp4a_ht_im
)
exp4a_dur_global_model = rmods.lmer(
    formula="log_hoverduration ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp4']),
    data=exp4a_ht_im
)
exp4b_hov_global_model = rmods.glmer(
    formula="hovered ~ (1 | sessionId) + (1 | grid) +"+"+".join(exp_analysis_predictors['exp4']),
    data=exp4b_ht_im
)
exp4b_dur_global_model = rmods.lmer(
    formula="log_hoverduration ~ (1 | sessionId) + (1 | grid) + "+"+".join(exp_analysis_predictors['exp4']),
    data=exp4b_ht_im
)
exp4a_hov_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4a Hover",
    data=exp4a_ht_im,
    dv="hovered",
    model_func="glmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)
exp4a_dur_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4a Hover Duration",
    data=exp4a_ht_im,
    dv="log_hoverduration",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)
exp4b_hov_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4b Hover",
    data=exp4b_ht_im,
    dv="hovered",
    model_func="glmer",
    random_effects="(1 | sessionId)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)
exp4b_dur_lesioning_res = analysisutils.global_lesion_analysis(
    name="Exp. 4b Hover Duration",
    data=exp4b_ht_im,
    dv="log_hoverduration",
    model_func="lmer",
    random_effects="(1 | sessionId) + (1 | grid)",
    predictors=exp_analysis_predictors['exp4'],
    rmods=rmods
)

100%|██████████| 11/11 [00:02<00:00,  4.29it/s]


## Create lesioned table

In [15]:
all_lesioning_res_static = [
    dict(res=exp1_lesioning_res, name=("Initial Exp", "Awareness")),
    dict(res=exp2_full_lesioning_res, name=("Up-front Exp", "Awareness (F)")),
    dict(res=exp2_early_lesioning_res, name=("Up-front Exp","Awareness (E)")),
    dict(res=exp3_correct_lesioning_res, name=("Critical Mazes Exp", "Accuracy")),
    dict(res=exp3_conf_lesioning_res, name=("Critical Mazes Exp", "Confidence")),
    dict(res=exp3_attn_lesioning_res, name=("Critical Mazes Exp", "Awareness")),
    dict(res=exp4a_hov_lesioning_res, name=("Process-Tracing Exp (Initial Mazes)", "Hovering")),
    dict(res=exp4a_dur_lesioning_res, name=("Process-Tracing Exp (Initial Mazes)", "Duration")),
    dict(res=exp4b_hov_lesioning_res, name=("Process-Tracing Exp (Critical Mazes)", "Hovering")),
    dict(res=exp4b_dur_lesioning_res, name=("Process-Tracing Exp (Critical Mazes)", "Duration")),
]

### Likelihood ratio test summary

In [16]:
all_summaries = []
for mod in all_lesioning_res_static:
    all_summaries.append(
        f"{', '.join(mod['name'])}: {mod['res'].likelihood_ratio_test('static_vgc_weight_Z', 1)}"
    )
all_summaries = '; \n'.join(all_summaries)
with open("./inputs/all_plan_exp_vgc_global_lesion_llr_svgc.tex", "w") as file:
    file.write(all_summaries)
print(all_summaries)

Initial Exp, Awareness: $\chi^2(1) = 400.63, p < 1.0 \times 10^{-16}$; 
Up-front Exp, Awareness (F): $\chi^2(1) = 203.67, p < 1.0 \times 10^{-16}$; 
Up-front Exp, Awareness (E): $\chi^2(1) = 127.79, p < 1.0 \times 10^{-16}$; 
Critical Mazes Exp, Accuracy: $\chi^2(1) = 86.07, p < 1.0 \times 10^{-16}$; 
Critical Mazes Exp, Confidence: $\chi^2(1) = 173.12, p < 1.0 \times 10^{-16}$; 
Critical Mazes Exp, Awareness: $\chi^2(1) = 376.52, p < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Initial Mazes), Hovering: $\chi^2(1) = 289.13, p < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Initial Mazes), Duration: $\chi^2(1) = 89.71, p < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Critical Mazes), Hovering: $\chi^2(1) = 210.47, p < 1.0 \times 10^{-16}$; 
Process-Tracing Exp (Critical Mazes), Duration: $\chi^2(1) = 242.78, p < 1.0 \times 10^{-16}$
